In [7]:
from __future__ import print_function
import sys
import re
import numpy as np

from numpy import dot
from numpy.linalg import norm
from pyspark import SparkContext
from operator import add
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.regression import LabeledPoint
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from time import time

Dimention=20000
smallTrainFile='SmallTrainingData.txt'
TestFile='TestingData.txt'

In [13]:
def freqArray (listOfIndices, numberofwords):
    returnVal = np.zeros (Dimention)
    for index in listOfIndices:
        returnVal[index] = returnVal[index] + 1
    returnVal = np.divide(returnVal, numberofwords)
    return returnVal


In [25]:
sc.stop()


# load training set
start_time = time()
sc = SparkContext(appName="MLlib_LR")

d_corpus = sc.textFile(smallTrainFile)
t_d_corpus = sc.textFile(TestFile)


numberOfDocs =d_corpus.count()

d_keyAndText = d_corpus.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))
regex = re.compile('[^a-zA-Z]')
d_keyAndListOfWords = d_keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

allWords = d_keyAndListOfWords.flatMap(lambda x: ((i,1) for i in x[1]))

allCounts = allWords.reduceByKey(lambda x,y: x+y)
allCounts.cache()
topWords = allCounts.top(Dimention, lambda x : x[1])

topWordsK = sc.parallelize(range(Dimention))
dictionary = topWordsK.map (lambda x : (topWords[x][0], x))

allWordsWithDocID = d_keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))

allDictionaryWords = dictionary.join (allWordsWithDocID)


justDocAndPos = allDictionaryWords.map (lambda x: (x[1][1], x[1][0]))

allDictionaryWordsInEachDoc = justDocAndPos.groupByKey()

allDocsAsNumpyArrays = allDictionaryWordsInEachDoc.map(lambda x: (x[0], freqArray(x[1],len(x[1]))))



# X_i
allDocsAsNumpyArrays.cache()
allDocsAsNumpyArrays.take(3)

[('AU466',
  array([2.95839346e-05, 1.81015855e-05, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00])),
 ('AU1356',
  array([2.41667689e-05, 1.28667866e-05, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00])),
 ('19264299',
  array([2.21781209e-05, 1.09011103e-05, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]))]

In [26]:
# load test dataset
t_numberOfDocs =t_d_corpus.count()

t_d_keyAndText = t_d_corpus.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))
regex = re.compile('[^a-zA-Z]')
t_d_keyAndListOfWords = t_d_keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

t_allWords = t_d_keyAndListOfWords.flatMap(lambda x: ((i,1) for i in x[1]))

t_allCounts = t_allWords.reduceByKey(lambda x,y: x+y)
t_allCounts.cache()
t_topWords = t_allCounts.top(Dimention, lambda x : x[1])

topWordsK = sc.parallelize(range(Dimention))
t_dictionary = topWordsK.map (lambda x : (t_topWords[x][0], x))

t_allWordsWithDocID = t_d_keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))

t_allDictionaryWords = t_dictionary.join (t_allWordsWithDocID)


t_justDocAndPos = t_allDictionaryWords.map (lambda x: (x[1][1], x[1][0]))

t_allDictionaryWordsInEachDoc = t_justDocAndPos.groupByKey()

t_allDocsAsNumpyArrays = t_allDictionaryWordsInEachDoc.map(lambda x: (x[0], freqArray(x[1],len(x[1]))))


# X_i
t_allDocsAsNumpyArrays.cache()
print(t_allDocsAsNumpyArrays.take(3))
load_data_duration=time() - start_time

[('13660744', array([1.81994631e-05, 7.85885908e-06, 0.00000000e+00, ...,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])), ('13915509', array([1.43312346e-05, 1.20465450e-05, 0.00000000e+00, ...,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])), ('15210035', array([1.25161484e-05, 6.74890353e-06, 0.00000000e+00, ...,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00]))]


In [27]:
def get_label(x):
    if 'AU' in x[0]:
        return 1
    else:
        return 0

In [37]:
Y_X_RDD=allDocsAsNumpyArrays.map(lambda x: LabeledPoint(get_label(x), x[1]))

In [38]:
train_start_time = time()
model = LogisticRegressionWithLBFGS.train(data=Y_X_RDD, iterations=200)
train_duration= time()-train_start_time

In [39]:
test_start_time=time()
prediction=model.predict(t_allDocsAsNumpyArrays.map(lambda x: x[1]))

In [40]:
prediction.cache()
prediction_list=prediction.collect()

In [41]:
label_list=[]
label_RDD=t_allDocsAsNumpyArrays.map(lambda x: 1 if 'AU' in x[0] else 0)
label_RDD.cache()

for i in label_RDD.collect():
    label_list.append(i)

In [42]:
print('f1 score:', f1_score(label_list, prediction_list, average='binary'))
print('confusion matrix:\n', confusion_matrix(label_list, prediction_list))
test_duration=time()-test_start_time
total_duration=time() - start_time

f1 score: 0.0
confusion matrix: [[18311    36]
 [  377     0]]


In [43]:
print(load_data_duration,train_duration,test_duration,total_duration)

231.1570689678192 33.947399854660034 9.641380071640015 8042.351692199707


In [44]:
prediction_list

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
